In [145]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [146]:
financial_data = pd.read_csv("BigPositionVsValue.csv")
match_data = pd.read_csv("MatchData.csv")

In [147]:
financial_data

,Year,Team,Position,Goal_Difference,Points,Squad_Size,Average_Age,Number_of_Foreigners,Average_Market_Value,Total_Market_Value
0,2017,manchester city,1,79,100,32,25.1,22,31.53,1010.00
1,2017,chelsea fc,5,40,81,46,24.7,33,19.22,884.25
2,2017,liverpool fc,4,38,77,35,24.9,21,24.50,857.50
3,2017,manchester united,2,46,75,35,26.1,22,24.27,849.50
4,2017,tottenham hotspur,3,24,70,34,24.7,22,24.40,829.60
...,...,...,...,...,...,...,...,...,...,...
115,2022,brentford fc,9,-30,38,36,24.8,26,10.31,371.20
116,2022,leeds united,19,-23,36,47,23.5,31,7.34,345.15
117,2022,crystal palace,11,-17,34,39,24.9,19,8.28,323.05
118,2022,fulham fc,10,-30,31,47,25.3,34,6.28,295.25


First, lets create a column which normalizes each teams Market_Value as a proportion of the most expensive team's market value for the year.   
  
In 2022, Manchester City had a market value of 1150 million pounds. This was the highest in the league. Their market value will now be 1.0. Every other team will have a market value which is:  
their market value / 1150.

In [148]:
relativeValue = []

#Loop through each row
for index, team in financial_data.iterrows():
    
    #Obtain which season we are looking at
    season = team['Year']
    
    #Create a new dataframe with just this season
    teamseason = financial_data[financial_data['Year'] == season]
    
    #Find the max value
    maxvalue = teamseason['Total_Market_Value'].max()
    
    #Divide this row's value by the max value for the season
    tempRelativeValue = team['Total_Market_Value']/maxvalue
    
    #Append it to our list
    relativeValue.append(tempRelativeValue)

#Add list to new column in main dataframe
financial_data["Normalized_Market_Value"] = relativeValue

#view the new table
financial_data

,Year,Team,Position,Goal_Difference,Points,Squad_Size,Average_Age,Number_of_Foreigners,Average_Market_Value,Total_Market_Value,Normalized_Market_Value
0,2017,manchester city,1,79,100,32,25.1,22,31.53,1010.00,1.000000
1,2017,chelsea fc,5,40,81,46,24.7,33,19.22,884.25,0.875495
2,2017,liverpool fc,4,38,77,35,24.9,21,24.50,857.50,0.849010
3,2017,manchester united,2,46,75,35,26.1,22,24.27,849.50,0.841089
4,2017,tottenham hotspur,3,24,70,34,24.7,22,24.40,829.60,0.821386
...,...,...,...,...,...,...,...,...,...,...,...
115,2022,brentford fc,9,-30,38,36,24.8,26,10.31,371.20,0.322783
116,2022,leeds united,19,-23,36,47,23.5,31,7.34,345.15,0.300130
117,2022,crystal palace,11,-17,34,39,24.9,19,8.28,323.05,0.280913
118,2022,fulham fc,10,-30,31,47,25.3,34,6.28,295.25,0.256739


In [149]:
match_data

,Year,Position,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90
0,22,1,Manchester City,38,28,5,5,94,33,61,89,2.34,78.6,32.1,46.5,1.22
1,22,2,Arsenal,38,26,6,6,88,43,45,84,2.21,71.9,42.0,29.9,0.79
2,22,3,Manchester Utd,38,23,6,9,58,43,15,75,1.97,67.7,50.4,17.3,0.45
3,22,4,Newcastle Utd,38,19,14,5,68,33,35,71,1.87,72.0,39.6,32.4,0.85
4,22,5,Liverpool,38,19,10,9,75,47,28,67,1.76,72.6,50.9,21.7,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,17,16,Huddersfield,38,9,10,19,28,58,-30,37,0.97,31.3,48.5,-17.2,-0.45
116,17,17,Southampton,38,7,15,16,37,56,-19,36,0.95,39.9,45.2,-5.3,-0.14
117,17,18,Swansea City,38,8,9,21,28,56,-28,33,0.87,31.6,58.6,-27.0,-0.71
118,17,19,Stoke City,38,7,12,19,35,68,-33,33,0.87,36.5,62.5,-26.0,-0.69


The team names are different in the two datasets, which is a problem as that is meant to be a primary key in joining the tables. We need to fix that by ensuring they are the same

In [150]:
match_teams = match_data["Squad"].unique()
match_teams.sort()
match_teams

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Burnley', 'Cardiff City', 'Chelsea', 'Crystal Palace', 'Everton',
       'Fulham', 'Huddersfield', 'Leeds United', 'Leicester City',
       'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Norwich City', "Nott'ham Forest", 'Sheffield Utd', 'Southampton',
       'Stoke City', 'Swansea City', 'Tottenham', 'Watford', 'West Brom',
       'West Ham', 'Wolves'], dtype=object)

In [151]:
financial_teams = financial_data["Team"].unique()
financial_teams.sort()
financial_teams

array(['afc bournemouth', 'arsenal fc', 'aston villa', 'brentford fc',
       'brighton & hove albion', 'burnley fc', 'cardiff city',
       'chelsea fc', 'crystal palace', 'everton fc', 'fulham fc',
       'huddersfield town', 'leeds united', 'leicester city',
       'liverpool fc', 'manchester city', 'manchester united',
       'newcastle united', 'norwich city', 'nottingham forest',
       'sheffield united', 'southampton fc', 'stoke city', 'swansea city',
       'tottenham hotspur', 'watford fc', 'west bromwich albion',
       'west ham united', 'wolverhampton wanderers'], dtype=object)

In [152]:
match_to_financial_map = {
    "Arsenal" : "arsenal fc", 
    "Aston Villa" : "aston villa",
    "Bournemouth" : "afc bournemouth",
    "Brentford" : "brentford fc",
    "Brighton" : "brighton & hove albion",
    "Burnley" : "burnley fc",
    "Cardiff City" : "cardiff city",
    "Chelsea" : "chelsea fc",
    "Crystal Palace" : "crystal palace",
    "Everton": "everton fc",
    "Fulham" : "fulham fc",
    "Huddersfield" : "huddersfield town",
    "Leeds United" : "leeds united", 
    "Leicester City" : "leicester city", 
    "Liverpool" : "liverpool fc",
    "Manchester City" : "manchester city",
    "Manchester Utd" : "manchester united",
    "Newcastle Utd" : "newcastle united",
    "Norwich City" : "norwich city",
    "Nott'ham Forest" : "nottingham forest",
    "Sheffield Utd" : "sheffield united",
    "Southampton" : "southampton fc",
    "Stoke City" : "stoke city",
    "Swansea City" : "swansea city",
    "Tottenham" : "tottenham hotspur",
    "Watford" : "watford fc",
    "West Brom" : "west bromwich albion",
    "West Ham" : "west ham united",
    "Wolves" : "wolverhampton wanderers"
}

In [153]:
for i in range(len(match_data)):
    if(match_data["Squad"][i] not in match_to_financial_map.values()):
        match_data["Squad"][i] = match_to_financial_map[match_data["Squad"][i]]

In [154]:
match_data

,Year,Position,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90
0,22,1,manchester city,38,28,5,5,94,33,61,89,2.34,78.6,32.1,46.5,1.22
1,22,2,arsenal fc,38,26,6,6,88,43,45,84,2.21,71.9,42.0,29.9,0.79
2,22,3,manchester united,38,23,6,9,58,43,15,75,1.97,67.7,50.4,17.3,0.45
3,22,4,newcastle united,38,19,14,5,68,33,35,71,1.87,72.0,39.6,32.4,0.85
4,22,5,liverpool fc,38,19,10,9,75,47,28,67,1.76,72.6,50.9,21.7,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,17,16,huddersfield town,38,9,10,19,28,58,-30,37,0.97,31.3,48.5,-17.2,-0.45
116,17,17,southampton fc,38,7,15,16,37,56,-19,36,0.95,39.9,45.2,-5.3,-0.14
117,17,18,swansea city,38,8,9,21,28,56,-28,33,0.87,31.6,58.6,-27.0,-0.71
118,17,19,stoke city,38,7,12,19,35,68,-33,33,0.87,36.5,62.5,-26.0,-0.69


In [155]:
financial_data

,Year,Team,Position,Goal_Difference,Points,Squad_Size,Average_Age,Number_of_Foreigners,Average_Market_Value,Total_Market_Value,Normalized_Market_Value
0,2017,manchester city,1,79,100,32,25.1,22,31.53,1010.00,1.000000
1,2017,chelsea fc,5,40,81,46,24.7,33,19.22,884.25,0.875495
2,2017,liverpool fc,4,38,77,35,24.9,21,24.50,857.50,0.849010
3,2017,manchester united,2,46,75,35,26.1,22,24.27,849.50,0.841089
4,2017,tottenham hotspur,3,24,70,34,24.7,22,24.40,829.60,0.821386
...,...,...,...,...,...,...,...,...,...,...,...
115,2022,brentford fc,9,-30,38,36,24.8,26,10.31,371.20,0.322783
116,2022,leeds united,19,-23,36,47,23.5,31,7.34,345.15,0.300130
117,2022,crystal palace,11,-17,34,39,24.9,19,8.28,323.05,0.280913
118,2022,fulham fc,10,-30,31,47,25.3,34,6.28,295.25,0.256739


Now the team names are synced and the primary keys are the same!

In [156]:
#Now lets sync the years
year_mapping = {
    17 : 2017,
    18 : 2018,
    19 : 2019,
    20 : 2020,
    21 : 2021,
    22 : 2022
}
for i in range(len(match_data)):
    if(match_data["Year"][i] in year_mapping.keys()):
        match_data["Year"][i] = year_mapping[match_data["Year"][i]]
        
match_data

,Year,Position,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90
0,2022,1,manchester city,38,28,5,5,94,33,61,89,2.34,78.6,32.1,46.5,1.22
1,2022,2,arsenal fc,38,26,6,6,88,43,45,84,2.21,71.9,42.0,29.9,0.79
2,2022,3,manchester united,38,23,6,9,58,43,15,75,1.97,67.7,50.4,17.3,0.45
3,2022,4,newcastle united,38,19,14,5,68,33,35,71,1.87,72.0,39.6,32.4,0.85
4,2022,5,liverpool fc,38,19,10,9,75,47,28,67,1.76,72.6,50.9,21.7,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2017,16,huddersfield town,38,9,10,19,28,58,-30,37,0.97,31.3,48.5,-17.2,-0.45
116,2017,17,southampton fc,38,7,15,16,37,56,-19,36,0.95,39.9,45.2,-5.3,-0.14
117,2017,18,swansea city,38,8,9,21,28,56,-28,33,0.87,31.6,58.6,-27.0,-0.71
118,2017,19,stoke city,38,7,12,19,35,68,-33,33,0.87,36.5,62.5,-26.0,-0.69


Now lets remove the unecessary rows from each data set!

In [157]:
match_data = match_data.drop(columns=["MP", "W", "D", "L", "GF", "GA", "GD", "Pts/MP"])
match_data

,Year,Position,Squad,Pts,xG,xGA,xGD,xGD/90
0,2022,1,manchester city,89,78.6,32.1,46.5,1.22
1,2022,2,arsenal fc,84,71.9,42.0,29.9,0.79
2,2022,3,manchester united,75,67.7,50.4,17.3,0.45
3,2022,4,newcastle united,71,72.0,39.6,32.4,0.85
4,2022,5,liverpool fc,67,72.6,50.9,21.7,0.57
...,...,...,...,...,...,...,...,...
115,2017,16,huddersfield town,37,31.3,48.5,-17.2,-0.45
116,2017,17,southampton fc,36,39.9,45.2,-5.3,-0.14
117,2017,18,swansea city,33,31.6,58.6,-27.0,-0.71
118,2017,19,stoke city,33,36.5,62.5,-26.0,-0.69


In [158]:
financial_data = financial_data.drop(columns=["Goal_Difference", "Squad_Size", "Number_of_Foreigners", "Average_Age"])

In [159]:
financial_data

,Year,Team,Position,Points,Average_Market_Value,Total_Market_Value,Normalized_Market_Value
0,2017,manchester city,1,100,31.53,1010.00,1.000000
1,2017,chelsea fc,5,81,19.22,884.25,0.875495
2,2017,liverpool fc,4,77,24.50,857.50,0.849010
3,2017,manchester united,2,75,24.27,849.50,0.841089
4,2017,tottenham hotspur,3,70,24.40,829.60,0.821386
...,...,...,...,...,...,...,...
115,2022,brentford fc,9,38,10.31,371.20,0.322783
116,2022,leeds united,19,36,7.34,345.15,0.300130
117,2022,crystal palace,11,34,8.28,323.05,0.280913
118,2022,fulham fc,10,31,6.28,295.25,0.256739


Now lets join both the tables to create a final data set to perform modelling

In [160]:
final_df = pd.merge(match_data, financial_data, left_on= ['Year', 'Squad'], right_on=['Year', 'Team'])

In [161]:
final_df

,Year,Position_x,Squad,Pts,xG,xGA,xGD,xGD/90,Team,Position_y,Points,Average_Market_Value,Total_Market_Value,Normalized_Market_Value
0,2022,1,manchester city,89,78.6,32.1,46.5,1.22,manchester city,1,89,33.70,1150.00,1.000000
1,2022,2,arsenal fc,84,71.9,42.0,29.9,0.79,arsenal fc,2,84,23.83,1000.00,0.869565
2,2022,3,manchester united,75,67.7,50.4,17.3,0.45,manchester united,3,71,17.67,848.00,0.737391
3,2022,4,newcastle united,71,72.0,39.6,32.4,0.85,newcastle united,4,61,14.25,541.60,0.470957
4,2022,5,liverpool fc,67,72.6,50.9,21.7,0.57,liverpool fc,5,67,20.82,811.85,0.705957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2017,16,huddersfield town,37,31.3,48.5,-17.2,-0.45,huddersfield town,16,31,3.03,106.05,0.105000
116,2017,17,southampton fc,36,39.9,45.2,-5.3,-0.14,southampton fc,17,47,10.24,297.00,0.294059
117,2017,18,swansea city,33,31.6,58.6,-27.0,-0.71,swansea city,18,36,4.60,165.49,0.163851
118,2017,19,stoke city,33,36.5,62.5,-26.0,-0.69,stoke city,19,41,5.50,192.45,0.190545


The final data frame has been created, now I will need to rearrange the column order to make it more readable. 

In [162]:
final_df = final_df.drop(columns=["Position_y", "Team", "Points"])

In [163]:
final_df

,Year,Position_x,Squad,Pts,xG,xGA,xGD,xGD/90,Average_Market_Value,Total_Market_Value,Normalized_Market_Value
0,2022,1,manchester city,89,78.6,32.1,46.5,1.22,33.70,1150.00,1.000000
1,2022,2,arsenal fc,84,71.9,42.0,29.9,0.79,23.83,1000.00,0.869565
2,2022,3,manchester united,75,67.7,50.4,17.3,0.45,17.67,848.00,0.737391
3,2022,4,newcastle united,71,72.0,39.6,32.4,0.85,14.25,541.60,0.470957
4,2022,5,liverpool fc,67,72.6,50.9,21.7,0.57,20.82,811.85,0.705957
...,...,...,...,...,...,...,...,...,...,...,...
115,2017,16,huddersfield town,37,31.3,48.5,-17.2,-0.45,3.03,106.05,0.105000
116,2017,17,southampton fc,36,39.9,45.2,-5.3,-0.14,10.24,297.00,0.294059
117,2017,18,swansea city,33,31.6,58.6,-27.0,-0.71,4.60,165.49,0.163851
118,2017,19,stoke city,33,36.5,62.5,-26.0,-0.69,5.50,192.45,0.190545


In [164]:
final_df = final_df.rename(columns={"Position_x": "Position", "Squad": "Team", "Pts": "Points"})

In [166]:
final_df

,Year,Position,Team,Points,xG,xGA,xGD,xGD/90,Average_Market_Value,Total_Market_Value,Normalized_Market_Value
0,2022,1,manchester city,89,78.6,32.1,46.5,1.22,33.70,1150.00,1.000000
1,2022,2,arsenal fc,84,71.9,42.0,29.9,0.79,23.83,1000.00,0.869565
2,2022,3,manchester united,75,67.7,50.4,17.3,0.45,17.67,848.00,0.737391
3,2022,4,newcastle united,71,72.0,39.6,32.4,0.85,14.25,541.60,0.470957
4,2022,5,liverpool fc,67,72.6,50.9,21.7,0.57,20.82,811.85,0.705957
...,...,...,...,...,...,...,...,...,...,...,...
115,2017,16,huddersfield town,37,31.3,48.5,-17.2,-0.45,3.03,106.05,0.105000
116,2017,17,southampton fc,36,39.9,45.2,-5.3,-0.14,10.24,297.00,0.294059
117,2017,18,swansea city,33,31.6,58.6,-27.0,-0.71,4.60,165.49,0.163851
118,2017,19,stoke city,33,36.5,62.5,-26.0,-0.69,5.50,192.45,0.190545


Download the new df as a final df

In [167]:
final_df.to_csv("Final_Data.csv")